In [ ]:
import numpy as np
path = "G:\\My Drive\\uni\\Machine Learning intro\\Introduction-To-Machine-Learning\\ex3\\mnist_all.npz"
np.set_printoptions(threshold=np.inf)


In [ ]:
data = np.load('mnist_all.npz')

def get_random_samples(digit, m):
    X = data[f'train{digit}']
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    samples = X[indices][:m]
    labels = np.full((m, 1), digit)
    return samples, labels


def get_small_sample(samples_per_digit):
    # make dictionary smaller sample from data
    X = {f"train{i}": get_random_samples(i, samples_per_digit)[
        0] for i in range(10)}
    return X


small_data = get_small_sample(30)

In [34]:
def cluster_dist(c1, c2):
    return np.array([np.linalg.norm(x1 - x2) for x1 in c1 for x2 in c2]).min()


def singlelinkage(X, k):
    """
    :param X: numpy array of size (m, d) containing the test samples
    :param k: the number of clusters
    :return: a column vector of length m, where C(i) ∈ {1, . . . , k} is the identity of the cluster in which x_i has been assigned.
    """
    m, d = X.shape
    clusters = np.arange(m)
    min_i, min_j = 0, 0
    curr_clusters_count = len(clusters)
    while curr_clusters_count > k:
        print (curr_clusters_count)
        distances = np.array([[cluster_dist(clusters[i], clusters[j]) for j in range(curr_clusters_count)] for i in range(curr_clusters_count)])
        np.fill_diagonal(distances, np.inf)
        min_i, min_j = np.unravel_index(np.argmin(distances), distances.shape)
        clusters[min_i] = np.concatenate((clusters[min_i], clusters[min_j]))
        clusters = np.delete(clusters, min_j, 0)
        curr_clusters_count -= 1
        print (clusters)



In [35]:
X = np.concatenate((small_data['train0'], small_data['train1']))
singlelinkage(X, k=10)

60


TypeError: 'numpy.int32' object is not iterable